In [1]:
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from keras.layers import Dense,Input
from keras import Sequential

In [3]:
# "hp" object is used as an argument to sample the hyperparameter range

In [4]:
def build_model(hp):
    units = hp.Int(name="units",min_value=16,max_value=64,step=16) # Range of units from 16 to 64
    model = Sequential([
        Dense(units,activation="relu"),
        Dense(10,activation="softmax")
    ])
    optimizer = hp.Choice(name="optimizer",values=["rmsprop","adam"])
    model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy"])
    return model

In [5]:
# More modular approach to build model

import keras_tuner as kt

In [6]:
class SimpleMLP(kt.HyperModel):
    def __init__(self,num_classes):
        self.num_classes = num_classes
    
    def build(self, hp):
        units = hp.Int(name="units",min_value=16,max_value=64,step=16) # Range of units from 16 to 64
        model = Sequential([
            Dense(units,activation="relu"),
            Dense(self.num_classes,activation="softmax")
        ])
        optimizer = hp.Choice(name="optimizer",values=["rmsprop","adam"])
        model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy"])
        return model

In [7]:
hypermodel = SimpleMLP(num_classes=10)

In [8]:
# Next step is to build a tuner(Using Bayesian Optmization here, which attempts to make smart predictions on the new hyper parameter values by looking at the previous choices)

tuner = kt.BayesianOptimization(build_model,objective="val_accuracy",max_trials=100,executions_per_trial=2,directory="mnist_kt_test",overwrite=True)

# Objective --> Metric that the tuner will seek to optimize
# Trials --> Maximum number of different model configurations to try before giving up
# Executions per trial --> To reduce metric variance, we can train the same model multiple times and average the results(How many trials to run for each model config)
# Directory --> Where to store the search logs
# Overwrite --> Whether to overwrite the data in directory to start a new search

In [9]:
# To get an overview of the search space

tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}


In [10]:
# For a non deafault metric, give direction="max" for accuracy related , and "min" for loss related

In [12]:
from keras.callbacks import EarlyStopping

In [14]:
(x_train,y_train),(x_test,y_test) = keras.datasets.mnist.load_data()

x_train = x_train.reshape((-1,28*28)).astype("float32") / 255
x_test = x_test.reshape((-1,28*28)).astype("float32") / 255
x_train_full = x_train[:]
x_test_full = x_test[:]
num_val_samples = 10000
x_train,x_val = x_train[:-num_val_samples] , x_train[-num_val_samples:]
y_train,y_val = y_train[:-num_val_samples], y_train[-num_val_samples:]
callbacks_list = [EarlyStopping(monitor="val_loss",patience=5)]
tuner.search(x_train,y_train,callbacks=callbacks_list,validation_data = [x_val, y_val],batch_size =128,epochs=100,verbose=2) # Similar to fit()

Trial 100 Complete [00h 00m 43s]
val_accuracy: 0.9726499915122986

Best val_accuracy So Far: 0.9763000011444092
Total elapsed time: 01h 44m 21s


In [15]:
top_n = 4
best_hps = tuner.get_best_hyperparameters(top_n) # To get the list of hyperparameters which we can padd suring model building
# best_model = tuner.get_best_models(top_n) # To get the list of best models 

In [16]:
# Once we get the best model, since we dont have to change the hyperparameters, we can include the validation data as well in the training process

In [17]:
# Now the last hyperparameter that we need to find : Number of epochs

In [23]:
def get_best_epoch(hp):
    model = build_model(hp)
    callbacks_list = [
        EarlyStopping(monitor="val_loss",patience=10) # We give patience as 10(high value since we dont need underfit models)
    ]
    history = model.fit(x_train,y_train,validation_data=[x_val,y_val],epochs=100,batch_size=128,callbacks=callbacks_list)
    val_loss_per_epoch = history.history["val_loss"]
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1 # To get the epoch where we had the lowest val loss
    print(f"Best epoch: {best_epoch}")
    return best_epoch,model

In [19]:
(x_train,y_train),(x_test,y_test) = keras.datasets.mnist.load_data()

x_train = x_train.reshape((-1,28*28)).astype("float32") / 255
x_test = x_test.reshape((-1,28*28)).astype("float32") / 255
x_train_full = x_train[:]
y_train_full = y_train[:]

In [24]:
def get_best_trained_model(hp):
    best_epoch ,model = get_best_epoch(hp)
    model.fit(x_train_full,y_train_full,batch_size=128,epochs=int(best_epoch*1.2))
    return model

In [25]:
best_models = [] 
for hp in best_hps:
    model = get_best_trained_model(hp)
    model.evaluate(x_test,y_test)
    best_models.append(model)

Epoch 1/100
469/469 [==============================] - 2s 4ms/step - loss: 0.3892 - accuracy: 0.8935 - val_loss: 0.2008 - val_accuracy: 0.9458
Epoch 2/100
469/469 [==============================] - 2s 4ms/step - loss: 0.1929 - accuracy: 0.9452 - val_loss: 0.1430 - val_accuracy: 0.9610
Epoch 3/100
469/469 [==============================] - 2s 3ms/step - loss: 0.1460 - accuracy: 0.9577 - val_loss: 0.1214 - val_accuracy: 0.9657
Epoch 4/100
469/469 [==============================] - 2s 3ms/step - loss: 0.1190 - accuracy: 0.9656 - val_loss: 0.0950 - val_accuracy: 0.9736
Epoch 5/100
469/469 [==============================] - 2s 4ms/step - loss: 0.1006 - accuracy: 0.9706 - val_loss: 0.0883 - val_accuracy: 0.9742
Epoch 6/100
469/469 [==============================] - 2s 4ms/step - loss: 0.0877 - accuracy: 0.9747 - val_loss: 0.0731 - val_accuracy: 0.9790
Epoch 7/100
469/469 [==============================] - 2s 4ms/step - loss: 0.0772 - accuracy: 0.9775 - val_loss: 0.0622 - val_accuracy: 0.9820

KeyboardInterrupt: 

In [ ]:
# Full run was taking lots of time on personal GPU, so stopped manually(No code error)